<a href="https://colab.research.google.com/github/BrunnaMarques/google_earth_engine/blob/main/GEE_Python_C%C3%A1lculo_de_%C3%8Dndices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Python code employs Google Earth Engine to compute three indices for an area in Pantanal, Mato Grosso state, Brazil. These indices, utilizing infrared bands, are commonly used for analyzing burned areas, identifying water bodies, and assessing vegetation health.


# Normalized Burn Ratio (NBR):

NBR utilizes bands from the near-infrared and shortwave infrared spectrums.
Key Bands:
Near-Infrared (NIR) Reflectance Band (typically around 850 nanometers).
Shortwave Infrared (SWIR) Reflectance Band (usually within 1,600 to 2,400 nanometers).
Formula:
NBR = (NIR + SWIR) / (NIR - SWIR)
NBR detects changes in vegetation, including those caused by factors like wildfires or droughts.

# NDWI (Normalized Difference Water Index):

Objective: Highlight water bodies, ideal for mapping lakes, rivers, and reservoirs.
Formula: NDWI = (NIR - SWIR) / (NIR + SWIR), with NIR representing Near-Infrared and SWIR representing Shortwave Infrared.
Interpretation: Higher NDWI values signify water presence, while lower values indicate non-water areas like land or vegetation.

# NDVI (Normalized Difference Vegetation Index):

Objective: Quantify vegetation density and health, commonly used in assessing plant growth, land cover changes, and agricultural monitoring.
Formula: NDVI = (NIR - Red) / (NIR + Red), with NIR as Near-Infrared and Red as the Red band.
Interpretation: NDVI values range from -1 to +1. High positive values (close to +1) indicate robust, dense vegetation, while low or negative values suggest non-vegetated surfaces or stressed vegetation.


In [1]:
##!pip list #verificar as bibliotecas

In [2]:
!pip install geemap earthengine-api -q  #pra não listar muitas linhas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.1 MB/s eta 0:00:00


In [3]:
#importar as bibliotecas
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display

In [ ]:
## autenticação do GEE. só da pra usar se tiver a conta
ee.Authenticate()
ee.Initialize()

In [5]:
## definir um map pra visualizar
Map = geemap.Map()
Map.split_map(left_layer = 'HYBRID', right_layer = 'ROADMAP')
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [6]:
##link gerado no code editor a partir de um ponto
roi = ee.Geometry.Point([-57.84158590350688, -20.336570895941065])

##Minha image collection
## todas as regiões dessa coleção da imagem vão ser selecionadas se intersectam com a roi  ##para filtrar imagens que possuem um certo percentual de cobertura de nuvens. o CLOUDY_PIXEL_PERCENTAGE foi copiado das propriedades do sentinel no dataset do GEE. e 1 é a porcentagem maxima
collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")\
                         .filterBounds(roi)\
                         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))\
                         .filterDate('2023-07-01', '2023-09-05')
##size visualiza todas as imagens. e get info pega essa informação
print('Quantas imagens eu tenho', collection.size().getInfo())

Quantas imagens eu tenho 4


In [7]:
##Selecionar a melhor imagem da colleção. PRODUCT_ID está descrito no dataset da imagem no code editor
melhor_imagem = collection.sort('CLOUDY_PIXEL_PERCENTAGE', True).first()
print('ID', melhor_imagem.get('PRODUCT_ID').getInfo())
print('% de nuvens', melhor_imagem.get('CLOUDY_PIXEL_PERCENTAGE').getInfo())


ID S2B_MSIL1C_20230720T135709_N0509_R067_T21KVT_20230720T172018
% de nuvens 0


In [8]:
##visualizar a imagem

## aplicação do fator de escala. cada coleção vai ter um valor de reflectancia associado a banda. Disponível nas caracteristicas do satelite no dataset
image_scale = melhor_imagem.select('B.*').multiply(0.0001)

##AddLayer
Map.addLayer(image_scale,{'bands':['B4', 'B3', 'B2'], 'min': 0.019, 'max': 0.19},'RGB Sentinel')
Map.addLayer(image_scale,{'bands':['B8', 'B4', 'B3'], 'min': 0.019, 'max': 0.19},'False Color Sentinel')
Map


Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [9]:
##Aplicar esse indice. calcular o NBR = permite ver as questões relacionadas a queimadas. indice normalizado. tem uma fórmula para o calculo
##NBR = (B08 - B12)/(B08+B12). dentro do GEE essa formula é simplificada assim: normalizedDifference()

nbr = image_scale.normalizedDifference(['B8', 'B12']).rename('nbr')
ndvi = image_scale.normalizedDifference(['B5', 'B4']).rename('ndvi')
ndwi = image_scale.normalizedDifference(['B8', 'B3']).rename('ndwi')


Map.addLayer(ndvi, {'palette':['red', 'orange', 'white', 'green'], 'min': -1, 'max': 1}, 'NDVI')
Map.addLayer(nbr, {'palette':['red','orange','yellow', 'green', 'darkgreen'], 'min': -1, 'max': 1}, 'NBR')
Map.addLayer(ndwi, {'palette':['darkblue', 'blue', 'yellow', 'white'], 'min': -1, 'max': 1}, 'NDWI')

left_layer = geemap.ee_tile_layer(image_scale,{'bands':['B4', 'B3', 'B2'], 'min': 0.019, 'max': 0.19},'RGB Sentinel')
right_layer = geemap.ee_tile_layer(nbr, {'palette':['red','orange','yellow', 'green', 'darkgreen'], 'min': -1.5, 'max': 1}, 'NBR')
Map.split_map(left_layer, right_layer)
Map


Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…